In [ ]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [ ]:
# Build the features
from src.features import build_features
(X,molecules)=build_features.build()

In [ ]:
# Build the targets
from src.features import build_targets
(y,aptamers)=build_targets.build_fold(molecules)


In [ ]:
# Create train/test sets
from src.models import split_data
X_train, X_test, y_train, y_test = split_data.split(0.4,X,y)

In [ ]:
# Train model
from src.models import train_model
model=train_model.train_rfc(X_train,y_train)

In [ ]:
# Test model
from src.models import predict_model
yp_train=predict_model.predict(model,X_train,y_train)
yp_test=predict_model.predict(model,X_test,y_test)

